### Process KYCOVID-19 confirmed cases from daily press releases

Data Source: https://governor.ky.gov/news  &  https://governor.ky.gov/attachments/COVID19_Case-Information.pdf

This notebook processes the PDF documents released by the Govenor of KY from 3-21-2020 to 3-24-2020.

In [1]:
import tika
from tika import parser
import pandas as pd
from pathlib import Path
from datetime import datetime

#### Setup variables to use

In [2]:
start_date = 20200321  #3-21 is the start date that the pdf became consistent
# establish the end_date based on the time of day. Anything prior to 6pm is previous day.
#end_date = int(datetime.now().strftime("%Y_%m_%d")) if int(datetime.now().strftime("%H")) > 16 else int(datetime.now().strftime("%Y_%m_%d"))-1 
end_date = 20200324
current_date = datetime.now().strftime("%Y_%m_%d")
filePath = Path("data")   # the file path for data
file_path_string = '/Users/mark/Documents/github-public/covid-19/data/'

#### Function to process PDF to dataframe

In [3]:
def process_pdf(file_to_process, file_date):
    raw = parser.from_file(file_to_process)
    raw = raw['content'].lstrip().rstrip() # Remove leading and trailing spaces
    raw=raw.replace('\n\n','\n')  # replace all double newline characters with one

    # Turn the string into a list, breaking on newline character
    string_list = [x.split(',') for x in raw.split(' \n')]
    string_list = string_list[2:-2]
    #print (string_list[:10])

    # Convert the list of string lists into a list of lists
    new_list=[]
    new_string=''
    for indx, items in enumerate(string_list):
        new_string=new_string + (str(items).replace(' ', ',', 2))
    new_string = new_string.replace("'","").replace('][','\n').replace('[','')
    new_list = [x.split(',') for x in new_string.split('\n')]
    #print (new_list[:10])

    # Create a dataframe from the list of lists
    df = pd.DataFrame(new_list,columns=['Age','M/F','County'])

    # Write dataframe to CSV file
    file_name = file_date+'_ky.csv'
    file_out = filePath.joinpath(file_name)  # path and filename
    df.to_csv(file_out)  # output to csv

    print (file_date,'length of dataframe:',len(df))  

#### Main processing loop

In [4]:
for increment in range(start_date, end_date+1):
    file_to_process = file_path_string+str(increment)+'_ky_covid19_case_information.pdf'
    process_pdf(file_to_process,str(increment))

20200321 length of dataframe: 86
20200322 length of dataframe: 102
20200323 length of dataframe: 124
20200324 length of dataframe: 157
